In [259]:
# Dependencies and Setup
import pandas as pd


In [260]:
# identify the data files
sensor_data_20_path = 'Data/Parking_Sensor_Data_2020.csv'
sensor_data_19_path = 'Data/Parking_Sensor_Data_2019.csv'
sensors_path = 'Data/Parking_Bay_Sensors.csv'

# test data read
# sensor_data_20_path = 'Data/sensor_data_20.csv'
# sensor_data_19_path = 'Data/sensor_data_19.csv'

# Read the data 

# vehicle movement on each parking sensor, date, sensor id & duration
sensor_data_20_df = pd.read_csv(sensor_data_20_path)
sensor_data_19_df = pd.read_csv(sensor_data_19_path)

# sensor location (lat & lon) and the unique id
sensors_df = pd.read_csv(sensors_path)


In [247]:
#Create test data 
# output_data_file = 'Data/sensor_data_19.csv'
# test_data_19 = sensor_data_19_df[(sensor_data_19_df['BayId'] == 3689) | (sensor_data_19_df['BayId'] == 6749)]
# test_data_19.to_csv(output_data_file, encoding='utf-8', index=False)

# output_data_file = 'Data/sensor_data_20.csv'
# test_data_20 = sensor_data_20_df[(sensor_data_20_df['BayId'] == 3042) | (sensor_data_20_df['BayId'] == 3511)]
# test_data_20.to_csv(output_data_file, encoding='utf-8', index=False)


In [261]:
# rename the bay id column to macth up with the transaction data
sensors_df = sensors_df.rename(columns = {'bay_id' : 'BayId'} )

In [262]:
# drop unused columns to reduce the size
sensors_df = sensors_df.drop(columns=['status', 'location'])
sensor_data_20_df = sensor_data_20_df.drop(columns=['SignPlateID','Sign','AreaName','StreetId','StreetName',
                                                    'BetweenStreet1ID',
                                                    'BetweenStreet1','BetweenStreet2ID','BetweenStreet2','SideOfStreet',
                                                    'SideName','InViolation','VehiclePresent'])
sensor_data_19_df = sensor_data_19_df.drop(columns=['SignPlateID','Sign','AreaName','StreetId','StreetName',
                                                    'BetweenStreet1ID',
                                                    'BetweenStreet1','BetweenStreet2ID','BetweenStreet2','SideOfStreet','SideOfStreetCode',
                                                    'SideName','InViolation','VehiclePresent'])


In [152]:
#sensor_data_19_df
#sensor_data_20_df

In [263]:
# drop NAN rows from all 3 data sets
sensors_df = sensors_df.dropna()
sensor_data_20_df = sensor_data_20_df.dropna()
sensor_data_19_df = sensor_data_19_df.dropna()

In [264]:
# Select only rows with more than 2 minutes duration
sensor_data_20_df = sensor_data_20_df[(sensor_data_20_df['DurationMinutes'] > 2)]
sensor_data_19_df = sensor_data_19_df[(sensor_data_19_df['DurationMinutes'] > 2)]

In [265]:
# Create a new Date column and set the date format to DD-MM-YYYY
sensor_data_20_df['Date'] = pd.to_datetime(sensor_data_20_df['DepartureTime']).dt.strftime('%d-%m-%Y')
sensor_data_19_df['Date'] = pd.to_datetime(sensor_data_19_df['DepartureTime']).dt.strftime('%d-%m-%Y')

In [266]:
# group by date & bay id and count the number of records (DeviceId)
# drop unused column and rename the DeviceId to Vehicle Count

# 2020 data
sensor_data_20_gr_df = sensor_data_20_df.groupby(['Date','BayId'], as_index=False).count()
sensor_data_20_gr_df = sensor_data_20_gr_df.drop(columns=['ArrivalTime','DepartureTime','StreetMarker','DurationMinutes'])
sensor_data_20_gr_df = sensor_data_20_gr_df.rename(columns = {'DeviceId' : 'Vehicle Count'} )

# 2019 data
sensor_data_19_gr_df = sensor_data_19_df.groupby(['Date','BayId'], as_index=False).count()
sensor_data_19_gr_df = sensor_data_19_gr_df.drop(columns=['ArrivalTime','DepartureTime','StreetMarker','DurationMinutes'])
sensor_data_19_gr_df = sensor_data_19_gr_df.rename(columns = {'DeviceId' : 'Vehicle Count'} )

In [243]:
#sensor_data_20_gr_df
#sensor_data_19_gr_df

In [267]:
# concatenate 2019 & 2020 data in to one new df
sensor_data_df = pd.concat([sensor_data_20_gr_df,sensor_data_19_gr_df])

In [244]:
# sensor_data_df

In [268]:
# merged the concatenated transaction data with sensor master data linked on Bay ID
sensors_data_merged_df = pd.merge(sensors_df, sensor_data_df, on='BayId', how='inner')

In [ ]:
#sensors_data_merged_d

In [269]:
# save the detail data in to a csv
output_detail_file = 'Data/cbd_sensor_detail_data.csv'
sensors_data_merged_df.to_csv(output_detail_file, encoding='utf-8', index=False)

In [270]:
# create a summary df with date and the total vehicles
sensors_data_summary_df = sensors_data_merged_df.groupby(['Date']).agg({'Vehicle Count':'sum'}).reset_index()

In [ ]:
#sensors_data_summary_df

In [271]:
# save the summary data in to a csv
output_summary_file = 'Data/cbd_sensor_summary_data.csv'
sensors_data_summary_df.to_csv(output_summary_file, encoding='utf-8', index=False)